In [44]:
import requests

In [48]:
request = {"StartDate":"2015-12-01"}

In [ ]:
##########################################################################
# Input Data Check
##########################################################################

try:
    request_json = request
    if request_json and 'StartDate' in request_json:  
        startDate = datetime.strptime(request_json['StartDate'], '%Y-%m-%d').date()            
    else:
        startDate = get_max_game_date()
    if request_json and 'EndDate' in request_json:  
        endDate = datetime.strptime(request_json['EndDate'], '%Y-%m-%d').date() 
    else:
        endDate = (datetime.now() + timedelta(days=-1)).date()
except Exception as e:
    raise ValueError("Start & End dates must be in YYYY-MM-DD format") from e

# Distinct list of Months between start and end date
delta = endDate - startDate       # as timedelta

if delta.days < 0:
    raise ValueError("Start Date can't be before End Date") 

##########################################################################
# Get Distinct Months for schedule to scrape
##########################################################################

yearmonths = []
for i in range(delta.days + 1):
    r = {}
    day = startDate + timedelta(days=i)
    r['monthname'] = day.strftime('%B').lower()
    if day.month > 9:
        r['year'] = day.year + 1
    else:
        r['year'] = day.year
    if r not in yearmonths: 
        yearmonths.append(r)

In [49]:
data = {"StartDate":"2015-12-01","EndDate":"2015-12-31"}

In [50]:
url = 'https://us-central1-nba-predictions-dev.cloudfunctions.net/nba_basketball_reference_scraper'

In [47]:
respomse = requests.post(url, data)

<Response [408]>

In [1]:
form_dict = {'HomeTeam':'Atlanta Hawks', 'AwayTeam':'Boston Celtics'}

In [9]:
import pandas as pd
import numpy as np

def predicted_pointspread(form_dict):
    try:
        df = pd.DataFrame(form_dict, index=[0])
        pointspread = np.random.randint(-40,40)
        pointspread = 1 if pointspread == 0 else pointspread
        if pointspread > 0:
            winner = df['HomeTeam'][0]
            loser = df['AwayTeam'][0]
        else:
            winner = df['AwayTeam'][0]
            loser = df['HomeTeam'][0]
        return f'I predict the {winner} will beat the {loser} by {abs(pointspread)} points'
    except:
        return 'Sorry, there was a problem processing the data entered... Please go back and double check your entries, thanks!'
        

In [10]:
predicted_pointspread(form_dict)

'I predict the Boston Celtics will beat the Atlanta Hawks by 39 points'

In [5]:
df = pd.DataFrame(form_dict, index=[0])

In [8]:
df['HomeTeam'][0]

'Atlanta Hawks'

In [12]:
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
from google.cloud import bigquery
import pyarrow

request = {"StartDate":"2015-07-01","EndDate":"2020-02-24"}

In [40]:
def get_max_game_date():
    client = bigquery.Client(project=projcet)
    
    QUERY = (
    "SELECT date_add(max(game_date), INTERVAL 1 day) as max_game_date FROM `nba.raw_basketballreference_game`"
    )
    query_job = client.query(QUERY)  # API request
    rows = query_job.result()  # Waits for query to finish
    
    for result in rows:
        load_date = result.max_game_date
    
    return load_date

def get_game_players(soup, player_game_data, id_string, game_key, stat_type, h_or_a, team_abbrev, game_date):
    rows = soup.find('table', id=id_string).find('tbody').find_all('tr')
    cnt = 1

    #print(str(rows))
    for player in rows:
        game_players = {}
        game_players['game_key'] = game_key
        game_players['game_date'] = game_date
        game_players['h_or_a'] = h_or_a
        game_players['team_abbrev'] = team_abbrev
        game_players['stat_period'] = stat_type
        game_players['player'] = player.find('th',{"data-stat": "player"}).text
        #print(game_players['player'])
        
        player_node = player.find('th',{"data-stat": "player"})
        
        # Ignore Header Line
        if game_players['player'] != 'Reserves' and player_node.has_attr('data-append-csv'):
        
            a = player.find('th',{"data-stat": "player"}).find('a',href=True)
            if a is not None:
                game_players['player_link'] = a['href']
            else:
                game_players['player_link'] = None
            
            game_players['player_key'] = player_node['data-append-csv']
            game_players['reason'] = get_text(player.find('td',{"data-stat": "reason"}))
            game_players['mp'] = get_text(player.find('td',{"data-stat": "mp"}))
            game_players['fg'] = get_text(player.find('td',{"data-stat": "fg"}))
            game_players['fga'] = get_text(player.find('td',{"data-stat": "fga"}))
            game_players['fg_pct'] = get_text(player.find('td',{"data-stat": "fg_pct"}))
            game_players['fg3'] = get_text(player.find('td',{"data-stat": "fg3"}))
            game_players['fg3a'] = get_text(player.find('td',{"data-stat": "fg3a"}))
            game_players['fg3_pct'] = get_text(player.find('td',{"data-stat": "fg3_pct"}))
            game_players['ft'] = get_text(player.find('td',{"data-stat": "ft"}))
            game_players['fta'] = get_text(player.find('td',{"data-stat": "fta"}))
            game_players['ft_pct'] = get_text(player.find('td',{"data-stat": "ft_pct"}))
            game_players['orb'] = get_text(player.find('td',{"data-stat": "orb"}))
            game_players['drb'] = get_text(player.find('td',{"data-stat": "drb"}))
            game_players['trb'] = get_text(player.find('td',{"data-stat": "trb"}))
            game_players['ast'] = get_text(player.find('td',{"data-stat": "ast"}))
            game_players['stl'] = get_text(player.find('td',{"data-stat": "stl"}))
            game_players['blk'] = get_text(player.find('td',{"data-stat": "blk"}))
            game_players['tov'] = get_text(player.find('td',{"data-stat": "tov"}))
            game_players['pf'] = get_text(player.find('td',{"data-stat": "pf"}))
            game_players['pts'] = get_text(player.find('td',{"data-stat": "pts"}))
            game_players['plus_minus'] = get_text(player.find('td',{"data-stat": "plus_minus"}))
            game_players['player_stat_key'] = game_players['game_key'] + '|' + game_players['player_key'] + '|' + game_players['stat_period'] 
            if cnt <= 5:
                game_players['starter_flag'] = True 
            else:
                game_players['starter_flag'] = False
            game_players['load_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  
            #print(game_players)
            player_game_data.append(game_players)
            cnt += 1

    return player_game_data
   
def get_text(stat):
    if stat is not None:
        if stat.text != "":
            txt = stat.text
        else:
            txt = None
    else:
        txt = None
    return txt

In [14]:
    
    ##########################################################################
    # Input Data Check
    ##########################################################################
    
    try:
        request_json = request
        if request_json and 'StartDate' in request_json:  
            startDate = datetime.strptime(request_json['StartDate'], '%Y-%m-%d').date()            
        else:
            startDate = get_max_game_date()
        if request_json and 'EndDate' in request_json:  
            endDate = datetime.strptime(request_json['EndDate'], '%Y-%m-%d').date() 
        else:
            endDate = (datetime.now() + timedelta(days=-1)).date()
    except Exception as e:
        raise ValueError("Start & End dates must be in YYYY-MM-DD format") from e

    # Distinct list of Months between start and end date
    delta = endDate - startDate       # as timedelta

    if delta.days < 0:
        raise ValueError("Start Date can't be before End Date") 
    
    ##########################################################################
    # Get Distinct Months for schedule to scrape
    ##########################################################################
    
    yearmonths = []
    for i in range(delta.days + 1):
        r = {}
        day = startDate + timedelta(days=i)
        r['monthname'] = day.strftime('%B').lower()
        if day.month > 9:
            r['year'] = day.year + 1
        else:
            r['year'] = day.year
        if r not in yearmonths: 
            yearmonths.append(r)
    #print(yearmonths)

In [20]:
yearmonths

[{'monthname': 'july', 'year': 2015},
 {'monthname': 'august', 'year': 2015},
 {'monthname': 'september', 'year': 2015},
 {'monthname': 'october', 'year': 2016},
 {'monthname': 'november', 'year': 2016},
 {'monthname': 'december', 'year': 2016},
 {'monthname': 'january', 'year': 2016},
 {'monthname': 'february', 'year': 2016},
 {'monthname': 'march', 'year': 2016},
 {'monthname': 'april', 'year': 2016},
 {'monthname': 'may', 'year': 2016},
 {'monthname': 'june', 'year': 2016},
 {'monthname': 'july', 'year': 2016},
 {'monthname': 'august', 'year': 2016},
 {'monthname': 'september', 'year': 2016},
 {'monthname': 'october', 'year': 2017},
 {'monthname': 'november', 'year': 2017},
 {'monthname': 'december', 'year': 2017},
 {'monthname': 'january', 'year': 2017},
 {'monthname': 'february', 'year': 2017},
 {'monthname': 'march', 'year': 2017},
 {'monthname': 'april', 'year': 2017},
 {'monthname': 'may', 'year': 2017},
 {'monthname': 'june', 'year': 2017},
 {'monthname': 'july', 'year': 2017}

In [22]:
yearmonths[3]

{'monthname': 'october', 'year': 2016}

In [24]:
    
##########################################################################
# Scrape Schedule
##########################################################################
player_game_rows_loaded = 0
game_rows_loaded = 0



schedule = []
v = yearmonths[3]
year = str(v['year'])
month = v['monthname']
url = 'https://www.basketball-reference.com/leagues/NBA_' + year + '_games-' + month + '.html'
print(url)

https://www.basketball-reference.com/leagues/NBA_2016_games-october.html


In [25]:
html = requests.get(url)

if html.ok:
    soup = BeautifulSoup(html.content, 'html.parser')  
else:
    print(f'No data for {month} {year} because enountered error code {html.status_code}')
    #continue

rows = soup.find('table', id="schedule").find('tbody').find_all('tr')
#print(rows)

for row in rows:
    game_date_node = row.find('th',{"data-stat": "date_game"})
    if game_date_node is not None:

        game_date = datetime.strptime(game_date_node.text, '%a, %b %d, %Y').date()
        if game_date >= startDate and game_date <= endDate:
            #cells = row.find_all(['td', 'th'])
            r = {}
            #r.setdefault(game_start_time, []).append(value)

            v1 = row.find('th',{"data-stat": "date_game"})
            #r[k1] = v1.text
            r['game_date'] = datetime.strptime(v1.text, '%a, %b %d, %Y').strftime("%Y-%m-%d")

            v2 = row.find('td',{"data-stat": "game_start_time"})
            r['game_start_time'] = v2.text if v2 else None

            v3 = row.find('td',{"data-stat": "visitor_team_name"})
            r['visitor_team_name'] = v3.text
            r['away_abbr'] = v3['csk'].split('.')[0]

            v4 = row.find('td',{"data-stat": "visitor_pts"})
            r['visitor_pts'] = v4.text if v4 else None

            v5 = row.find('td',{"data-stat": "home_team_name"})
            r['home_team_name'] = v5.text
            r['home_abbr'] = v5['csk'].split('.')[0]

            v6 = row.find('td',{"data-stat": "home_pts"})
            r['home_pts'] = v6.text if v6 else None

            v7 = row.find('td',{"data-stat": "box_score_text"}).find('a',href=True)
            r['box_score_url'] = v7['href'] if v7 else None

            v8 = row.find('td',{"data-stat": "attendance"})
            r['attendance'] = v8.text if v8 else None

            v9 = row.find('td',{"data-stat": "overtimes"})
            r['overtimes'] = v9.text if v9 else None

            if r['game_start_time']:
                v12 = r['away_abbr'] + r['game_date'].replace('-','') + r['home_abbr'] + r['game_start_time'].replace(':','')
            else:
                v12 = r['away_abbr'] + r['game_date'].replace('-','') + r['home_abbr']
            r['game_key'] = v12 if v12 else None

            schedule.append(r)
print(schedule)            

[{'game_date': '2015-10-27', 'game_start_time': '8:00p', 'visitor_team_name': 'Cleveland Cavaliers', 'away_abbr': 'CLE', 'visitor_pts': '95', 'home_team_name': 'Chicago Bulls', 'home_abbr': 'CHI', 'home_pts': '97', 'box_score_url': '/boxscores/201510270CHI.html', 'attendance': '21,957', 'overtimes': '', 'game_key': 'CLE20151027CHI800p'}, {'game_date': '2015-10-27', 'game_start_time': '8:00p', 'visitor_team_name': 'Detroit Pistons', 'away_abbr': 'DET', 'visitor_pts': '106', 'home_team_name': 'Atlanta Hawks', 'home_abbr': 'ATL', 'home_pts': '94', 'box_score_url': '/boxscores/201510270ATL.html', 'attendance': '19,187', 'overtimes': '', 'game_key': 'DET20151027ATL800p'}, {'game_date': '2015-10-27', 'game_start_time': '10:30p', 'visitor_team_name': 'New Orleans Pelicans', 'away_abbr': 'NOP', 'visitor_pts': '95', 'home_team_name': 'Golden State Warriors', 'home_abbr': 'GSW', 'home_pts': '111', 'box_score_url': '/boxscores/201510270GSW.html', 'attendance': '19,596', 'overtimes': '', 'game_key

In [41]:
        
            ##########################################################################
            # Scrape Games in Schedule
            ##########################################################################
            games_data = []
            player_game_data = []

            for game in schedule:
                if 'box_score_url' in game and game['box_score_url'] != "" and game['box_score_url'] is not None:


                    url = "https://www.basketball-reference.com" + game['box_score_url']

                    #print(url)
                    r = requests.get(url)
                    #print('here2')
                    soup = BeautifulSoup(str(r.content).replace("<!--","").replace('-->',''), 'html.parser')

                    ##############################################
                    # Line Score
                    rows = soup.find('table', id="line_score").find_all('tr')

                    # Away Line Score
                    r_num = 1
                    for away in rows[2].find_all('td'):
                        test_strong = away.find('strong') # Strong represents the total score ... ignore
                        if test_strong is None and r_num < 7:
                            k='a_g' + str(r_num) + '_score'
                            game[k] = away.text if away.text != "" else None
                        r_num+=1

                    # Home Line Score
                    r_num = 1
                    for home in rows[3].find_all('td'):
                        test_strong = home.find('strong') # Strong represents the total score ... ignore
                        if test_strong is None and r_num < 7:
                            k='h_g' + str(r_num) + '_score'
                            game[k] = home.text if home.text != "" else None
                        r_num+=1    

                    ##############################################
                    # Four Facts
                    rows = soup.find('table', id="four_factors").find_all('tr')

                    # Away Four Factors
                    game['a_ff_pace'] = rows[2].find('td',{"data-stat": "pace"}).text
                    game['a_ff_efg_pct'] = rows[2].find('td',{"data-stat": "efg_pct"}).text
                    game['a_ff_tov_pct'] = rows[2].find('td',{"data-stat": "tov_pct"}).text
                    game['a_ff_orb_pct'] = rows[2].find('td',{"data-stat": "orb_pct"}).text
                    game['a_ff_ft_rate'] = rows[2].find('td',{"data-stat": "ft_rate"}).text
                    game['a_ff_off_rtg'] = rows[2].find('td',{"data-stat": "off_rtg"}).text

                    # Home Four Factors
                    game['h_ff_pace'] = rows[3].find('td',{"data-stat": "pace"}).text
                    game['h_ff_efg_pct'] = rows[3].find('td',{"data-stat": "efg_pct"}).text
                    game['h_ff_tov_pct'] = rows[3].find('td',{"data-stat": "tov_pct"}).text
                    game['h_ff_orb_pct'] = rows[3].find('td',{"data-stat": "orb_pct"}).text
                    game['h_ff_ft_rate'] = rows[3].find('td',{"data-stat": "ft_rate"}).text
                    game['h_ff_off_rtg'] = rows[3].find('td',{"data-stat": "off_rtg"}).text
                    game['load_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")        

                    #now = datetime.now() # current date and time
                    #now.strftime("%m/%d/%Y, %H:%M:%S")


                    #player_game_data = []
                    game_date = game['game_date']

                    ##############################################
                    # Game Box - Home
                    #box-WAS-q1-basic
                    stat_type = "game"
                    h_or_a = "h"
                    team_abbrev = game['home_abbr']
                    id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
                    player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

                    ##############################################
                    # Game Box - Away
                    #box-WAS-q1-basic
                    stat_type = "game"
                    h_or_a = "a"
                    team_abbrev = game['away_abbr']
                    id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
                    player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q1 Box - Home
#                     stat_type = "q1"
#                     h_or_a = "h"
#                     team_abbrev = game['home_abbr']
#                     id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q1 Box - Away
#                     stat_type = "q1"
#                     h_or_a = "a"
#                     team_abbrev = game['away_abbr']
#                     id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q2 Box - Home
#                     stat_type = "q2"
#                     h_or_a = "h"
#                     team_abbrev = game['home_abbr']
#                     id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q2 Box - Away
#                     stat_type = "q2"
#                     h_or_a = "a"
#                     team_abbrev = game['away_abbr']
#                     id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q3 Box - Home
#                     stat_type = "q3"
#                     h_or_a = "h"
#                     team_abbrev = game['home_abbr']
#                     id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q3 Box - Away
#                     stat_type = "q3"
#                     h_or_a = "a"
#                     team_abbrev = game['away_abbr']
#                     id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q4 Box - Home
#                     stat_type = "q4"
#                     h_or_a = "h"
#                     team_abbrev = game['home_abbr']
#                     id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q4 Box - Away
#                     stat_type = "q4"
#                     h_or_a = "a"
#                     team_abbrev = game['away_abbr']
#                     id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

                    games_data.append(game)

In [27]:
pandas_games_data = pd.DataFrame(games_data)

In [29]:
pandas_games_data['game_start_time']

0      8:00p
1      8:00p
2     10:30p
3      7:00p
4      7:30p
5      7:30p
6      7:30p
7      7:30p
8      7:30p
9      8:00p
10     8:00p
11     8:00p
12     8:00p
13    10:00p
14    10:00p
15    10:00p
16    10:30p
17     7:00p
18     8:00p
19    10:30p
20     7:00p
21     7:00p
22     7:00p
23     7:30p
24     7:30p
25     8:00p
26     8:00p
27     8:30p
28     9:00p
29     9:30p
30    10:00p
31    10:30p
32     7:00p
33     7:00p
34     7:30p
35     8:00p
36    10:00p
37    10:30p
Name: game_start_time, dtype: object

In [ ]:
            ##########################################################################
            # Check for empty game data
            ##########################################################################
            # Continue to next month if there were no games in the month starting at the start date
            if not games_data:
                continue                    
  

In [37]:
project = 'nba-predictions-dev'

In [43]:
  
            ##########################################################################
            # Save to BigQuery
            ##########################################################################

            # print(player_game_data)
            # print(games_data)

            # Config
            client = bigquery.Client(project=project)
            print(f'Loading data for {month} {year}')
            
            #player game data
            pandas_player_game_data = pd.DataFrame(player_game_data)
            pandas_player_game_data['game_date'] = pandas_player_game_data['game_date'].astype('datetime64[ns]')
            pandas_player_game_data['load_datetime'] = pandas_player_game_data['load_datetime'].astype('datetime64[ns]')
            job_config = bigquery.LoadJobConfig()
            job_config.autodetect='True'
            job_config.create_disposition = 'CREATE_IF_NEEDED'
            job_config.write_disposition = 'WRITE_APPEND'
             ## Set schema for specific columns where more information is needed (e.g. not NULLABLE or specific date/time)
            job_config.schema = [
                bigquery.SchemaField('player_stat_key','STRING', 'REQUIRED'),
                bigquery.SchemaField('game_date','DATE'),
                bigquery.SchemaField('load_datetime','TIMESTAMP'),
                bigquery.SchemaField('starter_flag','BOOL')
            ]
            job_config.time_partitioning = bigquery.TimePartitioning(
                type_=bigquery.TimePartitioningType.DAY,
                field="game_date")
            job_player = client.load_table_from_dataframe(pandas_player_game_data, 'nba.raw_basketballreference_playerbox' \
                                                          , job_config=job_config, project=project)
            player_result = job_player.result()
            player_message = (
                f'Job ID: {player_result.job_id} '
                f'was started {player_result.started} '
                f'and ended {player_result.ended} '
                f'loading {player_result.output_rows} row(s) '
                f'to {player_result.destination}')
            print(player_message)
            player_game_rows_loaded = player_game_rows_loaded + player_result.output_rows

            #game data
            pandas_games_data = pd.DataFrame(games_data)
            pandas_games_data['game_date'] = pandas_games_data['game_date'].astype('datetime64[ns]')
            pandas_games_data['load_datetime'] = pandas_games_data['load_datetime'].astype('datetime64[ns]')
            job_config = bigquery.LoadJobConfig()
            job_config.autodetect='True'
            job_config.create_disposition = 'CREATE_IF_NEEDED'
            job_config.write_disposition = 'WRITE_APPEND'
            ## Set schema for specific columns where more information is needed (e.g. not NULLABLE or specific date/time)
            job_config.schema = [
                bigquery.SchemaField('game_key','STRING', 'REQUIRED'),
                bigquery.SchemaField('game_date','STRING', 'REQUIRED'),
                bigquery.SchemaField('home_team_name','STRING', 'REQUIRED'),
                bigquery.SchemaField('home_abbr','STRING', 'REQUIRED'),
                bigquery.SchemaField('visitor_team_name','STRING', 'REQUIRED'),
                bigquery.SchemaField('away_abbr','STRING', 'REQUIRED'),
                bigquery.SchemaField('game_date','DATE'),
                bigquery.SchemaField('load_datetime','TIMESTAMP'),
            ]
            job_config.time_partitioning = bigquery.TimePartitioning(
                type_=bigquery.TimePartitioningType.DAY,
                field="game_date")
            job_game = client.load_table_from_dataframe(pandas_games_data, 'nba.raw_basketballreference_game', \
                                                        job_config=job_config, project=project)
            game_result = job_game.result()
            game_message = (
                f'Job ID: {game_result.job_id} '
                f'was started {game_result.started} '
                f'and ended {game_result.ended} '
                f'loading {game_result.output_rows} row(s) '
                f'to {game_result.destination}')
            print(game_message)
            game_rows_loaded = game_rows_loaded + game_result.output_rows

            print(f'Successfully loaded {player_game_rows_loaded} row(s) to raw_basketballreference_playerbox and {game_rows_loaded} to raw_basketballreference_game')

Loading data for october 2016
Job ID: d6e1550a-cf9f-4c71-9e94-7e30078d29c6 was started 2021-02-24 20:46:21.330000+00:00 and ended 2021-02-24 20:46:23.183000+00:00 loading 983 row(s) to nba-predictions-dev.nba.raw_basketballreference_playerbox
Job ID: 18777676-ad8a-4154-84eb-1e5b25641c1e was started 2021-02-24 20:46:24.144000+00:00 and ended 2021-02-24 20:46:26.142000+00:00 loading 38 row(s) to nba-predictions-dev.nba.raw_basketballreference_game
Successfully loaded 1966 row(s) to raw_basketballreference_playerbox and 38 to raw_basketballreference_game
